In [1]:
import re
import csv
import nltk
import keras
import string
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from nltk import FreqDist
from termcolor import colored
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from tensorflow.keras.layers import LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional, Embedding

#Classifiers
from sklearn.naive_bayes import MultinomialNB

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/train.csv'
test_set_path = '/kaggle/input/ag-news-sinhala/ag_news_sinhala/test.csv'

In [6]:
stop_words = '/kaggle/input/stop-words/stop words.txt'
stem_dictionary = '/kaggle/input/stem-dictionary/stem_dictionary.txt'

In [7]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

In [8]:
# Concatenate train and test data
df = pd.concat([train_set, test_set], ignore_index=True)

# Save the combined data to a new CSV file
df.to_csv('combined.csv', index=False)

In [9]:
df.head()

,Unnamed: 0,Class Index,Title (English),Description (English),Title (Sinhala),Description (Sinhala)
0,0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


In [10]:
df = df.drop(['Unnamed: 0', 'Title (English)', 'Description (English)'], axis=1).copy()

In [11]:
df.head()

,Class Index,Title (Sinhala),Description (Sinhala)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල..."


In [12]:
labels = ['World', 'Sports', 'Business & Economics', 'Science & Technology']

In [13]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [14]:
df['Cleaned Title'] = df['Title (Sinhala)'].apply(lambda x: clean_text(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]"
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]"
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


In [15]:
df['Cleaned Description'] = df['Description (Sinhala)'].apply(lambda x: clean_text(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක..."
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද..."


In [16]:
stopwordsreader = open('/kaggle/input/stop-words/stop words.txt', 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [17]:
def remove_stop_words(tokens):
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [18]:
df['Stopwords Removed (Title)'] = df['Cleaned Title'].apply(lambda x: remove_stop_words(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]"
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]"
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


In [19]:
df['Stopwords Removed (Description)'] = df['Cleaned Description'].apply(lambda x: remove_stop_words(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක..."
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද..."


In [20]:
stem_dict = open('/kaggle/input/stem-dictionary/stem_dictionary.txt', "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]

In [21]:
def stemming_words(tokens):
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [22]:
df['Stemming Words (Title)'] = df['Stopwords Removed (Title)'].apply(lambda x: stemming_words(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [23]:
df['Stemming Words (Description)'] = df['Stopwords Removed (Description)'].apply(lambda x: stemming_words(x))
df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [24]:
df.drop_duplicates(subset='Stemming Words (Title)', keep='first', inplace=True)

df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [25]:
df.drop_duplicates(subset='Stemming Words (Description)', keep='first', inplace=True)

df.head()

,Class Index,Title (Sinhala),Description (Sinhala),Cleaned Title,Cleaned Description,Stopwords Removed (Title),Stopwords Removed (Description),Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය.,"නිව් යෝර්ක් (රොයිටර්) - කෙටි විකුණුම්කරුවන්, ව...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...","[නිව්, යෝර්ක්, රොයිටර්, කෙටි, විකුණුම්කරුවන්, ...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්),රොයිටර් - පුද්ගලික ආයෝජන සමාගමක් වන Carlyle Gr...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිටර්]","[රොයිටර්, පුද්ගලික, ආයෝජන, සමාගමක්, වන, ආරක්ෂක...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් - බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබ...,"[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...","[තෙල්, සහ, ආර්ථිකය]","[රොයිටර්, බොරතෙල්, මිල, ඉහළ, යාම, සහ, ආර්ථිකය,...",තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,රොයිටර් - කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම්වල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...","[රොයිටර්, කැරලිකාර, මිලීෂියාවකට, යටිතල, පහසුකම...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"AFP - ඉරාක ලෝක තෙල් මිල, වාර්තා පෙරලීම සහ මුදල...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...","[ඉරාක, ලෝක, තෙල්, මිල, වාර්තා, පෙරලීම, සහ, මුද...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [27]:
df.drop(df.columns[[1, 2, 3, 4, 5, 6]], axis=1, inplace=True)
df.head()

,Class Index,Stemming Words (Title),Stemming Words (Description)
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...


In [28]:
df.columns = ['ClassIndex', 'Stemming Words (Title)', 'Stemming Words (Description)']

In [29]:
df['Stemming Words'] = df['Stemming Words (Title)'] + " " + df['Stemming Words (Description)']

df.head()

,ClassIndex,Stemming Words (Title),Stemming Words (Description),Stemming Words
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය,නිව් යෝර්ක් රොයිටර් කෙටි විකුණුම්කරුවන් වෝල් ස...,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර්,රොයිටර් පුද්ගලික ආයෝජන සමාගමක් වන ආරක්ෂක කර්මා...,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය,රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ ආර්ථිකය පිළිබඳ ...,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...,රොයිටර් කැරලිකාර මිලීෂියාවකට යටිතල පහසුකම් පහර...,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...,ඉරාක ලෝක තෙල් මිල වාර්තා පෙරලීම සහ මුදල් පසුම්...,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [30]:
df.drop(df.columns[[1, 2]], axis=1, inplace=True)
df.head()

,ClassIndex,Stemming Words
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [32]:
df.isnull().sum()

ClassIndex        0
Stemming Words    0
dtype: int64

In [34]:
df['ClassIndex'].value_counts()

1    29810
2    28599
3    27876
4    26380
Name: ClassIndex, dtype: int64

In [35]:
df.shape

(112665, 2)

In [36]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['Stemming Words'])

In [37]:
print(X.shape)

(112665, 6379)


In [39]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, df['ClassIndex'], test_size = 0.2,random_state = 42)

In [42]:
from sklearn.metrics import f1_score, accuracy_score # mectric that use

In [43]:
model = MultinomialNB()

model.fit(X_train, y_train)

MultinomialNB()

In [46]:
review = 'ශ්‍රී ලංකාව හා දකුණු අප්‍රිකාව අතර අද(10) පැවැත්වීමට නියමිත පළමු පන්දුවාර 20යි 20 ක්‍රිකට් තරගය ක්‍රීඩා කිරීමේ අවස්ථාව කුසල් ජනිත් පෙරේරාට අහිමි වී ඇති බව වාර්තා වනවා.'

commenttype = model.predict(vectorizer.transform([review]))[0]

if commenttype == '1':
    print('World Comment')
elif commenttype == '2':
     print('Sport Comment')
elif commenttype == '3':
     print('Business Comment')
elif commenttype == '4':
     print('Science Comment')
else:
    print('Other Comment')

Other Comment
